In [1]:
!pip install -q -U kaggle_environments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 6.7 MB/s eta 0:00:00


In [10]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем стратегию агента, который всегда выбирает "камень". В терминах числового представления это соответствует значению 0.

In [11]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Давайте попробуем учитывать прошлые действия соперника. Опишем агента, который повторяет действие оппонента с предыдущего хода.

In [12]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Используем функцию evaluate из библиотеки kaggle_environments, чтобы запустить наших агентов и провести эксперимент на заданном числе игр.

In [13]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[1, None]]

In [14]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[-99.0, 99.0]]

In [15]:
# Определяем агентов со стратегиями

# 1. Агент всегда играет "камень"
def always_rock(observation, configuration):
    return 0

# 2. Агент всегда играет "бумагу"
def always_paper(observation, configuration):
    return 1

# 3. Агент всегда играет "ножницы"
def always_scissors(observation, configuration):
    return 2

# 4. Агент с случайным выбором
def random_agent(observation, configuration):
    return random.choice([0, 1, 2])

# 5. Циклический агент
def cycle_agent(observation, configuration):
    return observation.step % 3

# 6. Антициклический агент
def anti_cycle_agent(observation, configuration):
    return (2 - observation.step % 3)

# 7. Имитация предыдущего хода противника
def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    return 0

# 8. Агент, играющий противоположность предыдущего хода противника
def counter_opponent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % 3
    return 0

# 9. Агент двойной камень, потом бумага
def double_rock_then_paper(observation, configuration):
    if observation.step % 3 < 2:
        return 0
    return 1

# 10. Агент двойная бумага, потом ножницы
def double_paper_then_scissors(observation, configuration):
    if observation.step % 3 < 2:
        return 1
    return 2

# 11. Последовательный случайный агент
def sequential_random(observation, configuration):
    if observation.step > 0 and random.random() < 0.7:
        return observation.lastOpponentAction
    return random.choice([0, 1, 2])

# 12. Агент с трендом — выбирает ход, чаще всего использованный противником
def trend_agent(observation, configuration):
    if observation.step == 0:
        return random.choice([0, 1, 2])

    # Анализ предыдущих ходов противника
    counts = [0, 0, 0]
    for move in observation['opponent_history']:
        counts[move] += 1
    most_common = counts.index(max(counts))
    return (most_common + 1) % 3

# Формируем словарь агентов
agents = {
    "Always Rock": always_rock,
    "Always Paper": always_paper,
    "Always Scissors": always_scissors,
    "Random Agent": random_agent,
    "Cycle Agent": cycle_agent,
    "Anti-cycle Agent": anti_cycle_agent,
    "Copy Opponent": copy_opponent,
    "Counter Opponent": counter_opponent,
    "Double Rock Then Paper": double_rock_then_paper,
    "Double Paper Then Scissors": double_paper_then_scissors,
    "Sequential Random": sequential_random,
    "Trend Agent": trend_agent
}

# Создаем функция для проведения турнира между агентами
def run_tournament(agents):
    # Создаем среду "камень-ножницы-бумага"
    env = make("rps", configuration={"episodeSteps": 100})
    results = {name: 0 for name in agents}  # Словарь для хранения результатов

    # Проводим турнир: каждый агент играет с каждым
    for agent1_name, agent1 in agents.items():
        for agent2_name, agent2 in agents.items():
            if agent1_name == agent2_name:
                continue  # Агент не играет сам с собой

            # Настройка окружения для игры между двумя агентами
            env.reset()
            # Запуск игры между agent1 и agent2
            outcome = env.run([agent1, agent2])
            final_state = outcome[-1]  # Последний шаг

            # Анализируем результат
            reward_1 = final_state[0]["reward"] if final_state[0]["reward"] is not None else 0
            reward_2 = final_state[1]["reward"] if final_state[1]["reward"] is not None else 0

            # Обновляем счет
            if reward_1 > reward_2:
                results[agent1_name] += 1
            elif reward_1 < reward_2:
                results[agent2_name] += 1

    return results

# Запускаем турнир и выводим результаты
if __name__ == "__main__":
    tournament_results = run_tournament(agents)
    # Сортируем результаты для наглядности
    sorted_results = sorted(tournament_results.items(), key=lambda x: x[1], reverse=True)

    print("Результаты турнира:")
    print(f"{'Место':<6} {'Агент':<20} {'Победы':<10}")
    print("-" * 40)
    for rank, (agent, score) in enumerate(sorted_results, start=1):
        print(f"{rank:<6} {agent:<20} {score:<10}")


Результаты турнира:
Место  Агент                Победы    
----------------------------------------
1      Counter Opponent     10        
2      Double Paper Then Scissors 10        
3      Cycle Agent          8         
4      Double Rock Then Paper 8         
5      Always Paper         6         
6      Always Scissors      6         
7      Anti-cycle Agent     6         
8      Always Rock          4         
9      Copy Opponent        4         
10     Sequential Random    4         
11     Random Agent         2         
12     Trend Agent          0         
